<a href="https://colab.research.google.com/github/mlaricobar/solution-datathon-interbank-2020/blob/desarrollo/%5B04%5D%20Model%20Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSTRUCCIÓN DE LOS MODELOS


### Instalación de Librerías necesarias

In [1]:
!pip install catboost

     |████████████████████████████████| 66.3MB 46kB/s 



### Lectura de Datos

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competencias/'[02] Interbank-2020'

/gdrive/My Drive/Competencias/[02] Interbank-2020


In [4]:
!ls -lh '[01] Intermediate'

total 216M
-rw------- 1 root root 81M Dec 17 14:57 ds_rcc.csv
-rw------- 1 root root 89M Dec 17 14:57 ds_rcc_test.csv
-rw------- 1 root root 23M Dec 17 15:01 ds_se.csv
-rw------- 1 root root 24M Dec 17 15:01 ds_se_test.csv


In [5]:
import numpy as np
import pandas as pd

In [6]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [7]:
df_se = optimizar_df(pd.read_csv('[01] Intermediate/ds_se.csv'))
df_rcc = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc.csv"))

df_se_test = optimizar_df(pd.read_csv('[01] Intermediate/ds_se_test.csv'))
df_rcc_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_test.csv"))

#### Consolidar Dataset

In [8]:
df_train = (df_se.merge(df_rcc, how="left", on=["key_value"])
                .reset_index(drop=True)
           )

df_test = (df_se_test.merge(df_rcc_test, how="left", on=["key_value"])
                .reset_index(drop=True)
           )

In [9]:
df_train.head()

,key_value,edad,sexo,est_cvl,sit_lab,cod_ocu,ctd_hijos,flg_sin_email,ctd_veh,cod_ubi,lgr_vot,prv,dto,rgn,tip_lvledu,target,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_RECENCY,U03M_LONG_RECENCY,U03M_DIFF_MONTHS_FIRST_LAST
0,0,-0.983651,1,4,2,22,0.0,1,0.0,-0.070167,17,47,687,4,0,0,0.062248,0.186743,3,-0.072355,-0.072355,1,-0.038181,-0.114542,3,-0.038442,-0.115326,3,NaN,0.000000,0,NaN,0.000000,0,-0.072120,-0.072120,1,NaN,0.0,0,NaN,0.000000,0,NaN,0.0,0,-0.013735,-0.041204,3,-0.062533,-0.187599,3,3.666667,11,3,1,3,2
1,1,0.000953,1,4,1,22,0.0,1,1.0,-0.738566,10,186,849,6,0,0,NaN,0.000000,0,-0.072717,-0.218150,3,NaN,0.000000,0,NaN,0.000000,0,-0.062238,-0.186714,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,-0.072881,-0.072881,1,NaN,0.0,0,-0.068062,-0.204186,3,-0.159249,-0.477746,3,2.333333,7,3,1,3,2
2,2,0.531124,0,4,1,22,0.0,1,0.0,-0.297248,15,84,1405,1,7,0,NaN,0.000000,0,-0.072758,-0.072758,1,NaN,0.000000,0,NaN,0.000000,0,-0.004746,-0.004746,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,-0.072881,-0.072881,1,NaN,0.0,0,-0.050128,-0.050128,1,-0.150384,-0.150384,1,3.000000,3,1,1,1,0
3,3,-1.135129,0,4,1,22,0.0,1,0.0,-1.203435,4,9,977,6,7,0,NaN,0.000000,0,-0.072791,-0.072791,1,NaN,0.000000,0,NaN,0.000000,0,0.095067,0.095067,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,0.095067,0.095067,1,NaN,0.0,0,0.039114,0.039114,1,0.117343,0.117343,1,3.000000,3,1,1,1,0
4,4,-1.438084,0,4,1,22,0.0,1,0.0,0.043424,18,111,391,3,0,1,-0.056247,-0.168741,3,-0.072403,-0.217210,3,-0.058336,-0.175007,3,-0.067921,-0.203762,3,NaN,0.000000,0,-0.070507,-0.211522,3,-0.063319,-0.189956,3,NaN,0.0,0,-0.061331,-0.183993,3,NaN,0.0,0,-0.064295,-0.192884,3,-0.450063,-1.350190,3,7.000000,21,3,1,3,2


In [10]:
df_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358428 entries, 0 to 358427
Data columns (total 58 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   key_value                                          358428 non-null  uint32 
 1   edad                                               354663 non-null  float32
 2   sexo                                               358428 non-null  uint8  
 3   est_cvl                                            358428 non-null  uint8  
 4   sit_lab                                            358428 non-null  uint8  
 5   cod_ocu                                            358428 non-null  uint8  
 6   ctd_hijos                                          358428 non-null  float32
 7   flg_sin_email                                      358428 non-null  uint8  
 8   ctd_veh                                            358428 non-null  float3

In [18]:
df_test.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396609 entries, 0 to 396608
Data columns (total 57 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   key_value                                          396609 non-null  uint32 
 1   edad                                               386975 non-null  float32
 2   sexo                                               396609 non-null  uint8  
 3   est_cvl                                            396609 non-null  uint8  
 4   sit_lab                                            396609 non-null  uint8  
 5   cod_ocu                                            396609 non-null  uint8  
 6   ctd_hijos                                          396609 non-null  float32
 7   flg_sin_email                                      396609 non-null  uint8  
 8   ctd_veh                                            396609 non-null  float3

In [13]:
for col_ in df_train.columns:
  if df_train.loc[df_train[col_] == np.inf, :].shape[0] > 0:
    print(col_)

In [16]:
avoid_columns = []
main_columns = ["key_value", "target"]
predictores = [c for c in df_train.columns if c not in (main_columns + avoid_columns)]

In [17]:
print(predictores)

['edad', 'sexo', 'est_cvl', 'sit_lab', 'cod_ocu', 'ctd_hijos', 'flg_sin_email', 'ctd_veh', 'cod_ubi', 'lgr_vot', 'prv', 'dto', 'rgn', 'tip_lvledu', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_LOW

### Preparar Marco de Evaluación

In [19]:
nrows, nrows_train, nrows_test = (df_train.shape[0] + df_test.shape[0]), df_train.shape[0], df_test.shape[0]

print("Cantidad de filas: {0:,}\n\t- Train:\t{1:,} filas\t({2:}%)\n\t- Test:\t\t{3:,} filas\t({4:}%)".format(nrows,
                                                                                                            nrows_train,
                                                                                                            round((100 * (nrows_train/nrows)), 2),
                                                                                                            nrows_test,
                                                                                                            round((100 * (nrows_test/nrows)), 2)
                                                                                                            ))

Cantidad de filas: 755,037
	- Train:	358,428 filas	(47.47%)
	- Test:		396,609 filas	(52.53%)


In [20]:
df_train.head()

,key_value,edad,sexo,est_cvl,sit_lab,cod_ocu,ctd_hijos,flg_sin_email,ctd_veh,cod_ubi,lgr_vot,prv,dto,rgn,tip_lvledu,target,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_RECENCY,U03M_LONG_RECENCY,U03M_DIFF_MONTHS_FIRST_LAST
0,0,-0.983651,1,4,2,22,0.0,1,0.0,-0.070167,17,47,687,4,0,0,0.062248,0.186743,3,-0.072355,-0.072355,1,-0.038181,-0.114542,3,-0.038442,-0.115326,3,NaN,0.000000,0,NaN,0.000000,0,-0.072120,-0.072120,1,NaN,0.0,0,NaN,0.000000,0,NaN,0.0,0,-0.013735,-0.041204,3,-0.062533,-0.187599,3,3.666667,11,3,1,3,2
1,1,0.000953,1,4,1,22,0.0,1,1.0,-0.738566,10,186,849,6,0,0,NaN,0.000000,0,-0.072717,-0.218150,3,NaN,0.000000,0,NaN,0.000000,0,-0.062238,-0.186714,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,-0.072881,-0.072881,1,NaN,0.0,0,-0.068062,-0.204186,3,-0.159249,-0.477746,3,2.333333,7,3,1,3,2
2,2,0.531124,0,4,1,22,0.0,1,0.0,-0.297248,15,84,1405,1,7,0,NaN,0.000000,0,-0.072758,-0.072758,1,NaN,0.000000,0,NaN,0.000000,0,-0.004746,-0.004746,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,-0.072881,-0.072881,1,NaN,0.0,0,-0.050128,-0.050128,1,-0.150384,-0.150384,1,3.000000,3,1,1,1,0
3,3,-1.135129,0,4,1,22,0.0,1,0.0,-1.203435,4,9,977,6,7,0,NaN,0.000000,0,-0.072791,-0.072791,1,NaN,0.000000,0,NaN,0.000000,0,0.095067,0.095067,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,0.095067,0.095067,1,NaN,0.0,0,0.039114,0.039114,1,0.117343,0.117343,1,3.000000,3,1,1,1,0
4,4,-1.438084,0,4,1,22,0.0,1,0.0,0.043424,18,111,391,3,0,1,-0.056247,-0.168741,3,-0.072403,-0.217210,3,-0.058336,-0.175007,3,-0.067921,-0.203762,3,NaN,0.000000,0,-0.070507,-0.211522,3,-0.063319,-0.189956,3,NaN,0.0,0,-0.061331,-0.183993,3,NaN,0.0,0,-0.064295,-0.192884,3,-0.450063,-1.350190,3,7.000000,21,3,1,3,2


In [21]:
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

from scipy.optimize import differential_evolution

In [22]:
seed_val = 41

ctb_params = {'loss_function':'Logloss',
              'eval_metric':'AUC',
              'task_type': 'GPU',
              'verbose': 200,
              'random_seed': 1
             }

lgb_params = {
                "objective": "binary",
                "metric": "auc",
                "max_depth": 8,
                "learning_rate": 0.05,
                "bagging_seed": 41,
                "min_data_in_leaf": 20,
                "bagging_fraction": 0.7,
                "feature_fraction": 0.7,
                "bagging_freq": 5,
                "verbosity": -1
              }

In [23]:
k = 5 # n folds

def model_with_cv(df_train, df_test, target_class, predictores, df_val=None):

  ##################################################################
  ############# ETAPA DE PREDICCIÓN
  ##################################################################

  # Se instancian las variables que se obtendrán como resultado.
  results_models = []
  list_models_cv = []

  df_probs_train = df_train[["key_value", target_class]].copy().reset_index(drop=True)
  df_probs_test = df_test[["key_value"]].copy().reset_index(drop=True)

  if df_val is not None:
    df_probs_val = df_val[["key_value", target_class]].copy().reset_index(drop=True)
  else:
    df_probs_val = None
  # Generar los Folds
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

  for i, (traincv, valcv) in enumerate(skf.split(df_train[predictores], df_train[target_class])):

      print("Iteracion {}".format(i))
      
      df_train_cv = df_train.loc[traincv, :].copy().reset_index(drop=True)
      df_val_cv = df_train.loc[valcv, :].copy().reset_index(drop=True)

      # Fase de Entrenamiento
      print("\t CLASIFICACIÓN")
      print("\t\t 1. CATBOOST")
      ## CatBoost
      cbt_model = CatBoostClassifier(**ctb_params)
      cbt_model.fit(df_train_cv[predictores], df_train_cv[target_class], eval_set=(df_val_cv[predictores], df_val_cv[target_class]), use_best_model=True)

      print("\t\t 2. LIGHTGBM")
      ## LightGBM
      lgtrain = lgb.Dataset(df_train_cv[predictores], label=df_train_cv[target_class])
      lgval = lgb.Dataset(df_val_cv[predictores], label=df_val_cv[target_class])
      lgb_model = lgb.train(lgb_params, lgtrain, 900, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=20, evals_result={})

      # Fase de Scoring
      ## Clasificación
      probas_train_cbtm = cbt_model.predict_proba(df_train_cv[predictores])[:, 1]
      probas_val_cbtm = cbt_model.predict_proba(df_val_cv[predictores])[:, 1]

      probas_train_lgbm = lgb_model.predict(df_train_cv[predictores])
      probas_val_lgbm = lgb_model.predict(df_val_cv[predictores])

      # Cálculo de Métricas
      ## Clasificación
      metric_train_cbtm = roc_auc_score(df_train_cv[target_class], probas_train_cbtm)
      metric_val_cbtm = roc_auc_score(df_val_cv[target_class], probas_val_cbtm)

      metric_train_lgbm = roc_auc_score(df_train_cv[target_class], probas_train_lgbm)
      metric_val_lgbm = roc_auc_score(df_val_cv[target_class], probas_val_lgbm)

      if df_val is not None:
        probas_val_out_cbtm = cbt_model.predict_proba(df_val[predictores])[:, 1]
        metric_val_out_cbtm = roc_auc_score(df_val[target_class], probas_val_out_cbtm)

        probas_val_out_lgbm = lgb_model.predict(df_val[predictores])
        metric_val_out_lgbm = roc_auc_score(df_val[target_class], probas_val_out_lgbm)

      else:
        probas_val_out_cbtm = np.zeros((1,1))
        metric_val_out_cbtm = 0

        probas_val_out_lgbm = np.zeros((1,1))
        metric_val_out_lgbm = 0

      # Cálculo de probabilidades en la base de Train y Validacion
      df_probs_train.loc[valcv, "prob_cbtm"] = probas_val_cbtm
      df_probs_train.loc[valcv, "prob_lgbm"] = probas_val_lgbm
      df_probs_train.loc[valcv, "FOLD"] = "FOLD_{0}".format(i+1)

      
      df_probs_test["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_test[predictores])[:, 1]
      df_probs_test["prob_lgbm_{}".format(i)] = lgb_model.predict(df_test[predictores])

      if df_val is not None:
        df_probs_val["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_val[predictores])[:, 1]
        df_probs_val["prob_lgbm_{}".format(i)] = lgb_model.predict(df_val[predictores])

        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, "auc_val_out_cbtm": metric_val_out_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, "auc_val_out_lgbm": metric_val_out_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4), round(df_val[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4), round(np.mean(probas_val_out_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4), round(np.std(probas_val_out_cbtm),4)))

      else:

        # Almacenar los resultados
        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4)))
        
        
      list_models_cv.append({"cbtm_model": cbt_model, "lgbm_model": lgb_model})
      
      print("CATBOOST REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_cbtm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_cbtm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)

      print("LIGHTGBM REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_lgbm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_lgbm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)
  
  ##################################################################
  ############# PARA ETAPA DE OPTIMIZACIÓN
  ##################################################################

  if df_val is not None:
    df_probs_val["prob_avg_cbtm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_cbtm" in c]].mean(axis=1)
    df_probs_val["prob_avg_lgbm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_lgbm" in c]].mean(axis=1)

  df_probs_test["prob_avg_cbtm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_cbtm" in c]].mean(axis=1)
  df_probs_test["prob_avg_lgbm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_lgbm" in c]].mean(axis=1)

  
  return results_models, list_models_cv, df_probs_train, df_probs_val, df_probs_test

##### Modelo de compra de TC

In [24]:
results, list_models, df_probs_train, df_probs_val, df_probs_test = model_with_cv(df_train, df_test, "target", predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.044019
0:	learn: 0.7199031	test: 0.7207297	best: 0.7207297 (0)	total: 13.4ms	remaining: 13.4s
200:	learn: 0.8139863	test: 0.8127122	best: 0.8127122 (200)	total: 2.32s	remaining: 9.21s
400:	learn: 0.8224136	test: 0.8184261	best: 0.8184261 (400)	total: 4.61s	remaining: 6.89s
600:	learn: 0.8273690	test: 0.8206585	best: 0.8206585 (600)	total: 6.92s	remaining: 4.59s
800:	learn: 0.8313113	test: 0.8219892	best: 0.8219905 (799)	total: 9.15s	remaining: 2.27s
999:	learn: 0.8345468	test: 0.8227076	best: 0.8227119 (998)	total: 11.4s	remaining: 0us
bestTest = 0.822711885
bestIteration = 998
Shrink model to first 999 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.79629	valid_1's auc: 0.795504
[40]	training's auc: 0.806211	valid_1's auc: 0.803961
[60]	training's auc: 0.812928	valid_1's auc: 0.809922
[80]	training's auc: 0.817591	valid_1's auc: 0.81382
[100]	training's au

In [25]:
# First Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.834539,0.822712,0.859006,0.823846,41
1,0.835161,0.820945,0.855816,0.821616,41
2,0.835441,0.817861,0.857260,0.819050,41
3,0.835025,0.821256,0.851961,0.822491,41
4,0.835012,0.819481,0.872939,0.821429,41


In [28]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")
    df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)

df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(30)

,name,importance_fold_0,importance_fold_1,importance_folds_mean,importance_fold_2,importance_fold_3,importance_fold_4
0,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,9.499236,12.185535,11.932321,12.533133,11.943451,13.947286
1,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,10.353953,9.815137,10.035404,10.291345,9.880182,9.776434
2,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,5.552965,5.930999,5.558967,5.209741,5.682209,5.380917
3,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,4.993789,4.932552,4.851282,4.740744,4.823784,4.737139
4,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,5.425970,4.507779,4.831557,4.685477,4.505515,5.060344
5,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,4.442908,4.983072,4.758920,4.433825,5.345147,4.578937
6,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,4.464960,4.231054,4.023983,3.817681,3.778862,3.749748
7,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,3.440408,3.003731,3.308666,3.961395,3.062636,3.008810
8,edad,3.252062,3.180748,3.189096,3.054243,3.273725,3.186472
9,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,3.318759,3.312456,3.185686,3.163105,3.082708,3.006184


In [ ]:
for i, pair_m in enumerate(list_models_renta_2):
  print("Modelo {0}".format(i))
  df_probs_train_renta_2_complement["prob_cbtm_{}".format(i)] = pair_m["cbtm_model"].predict_proba(df_probs_train_renta_2_complement[predictores])[:, 1]
  df_probs_train_renta_2_complement["prob_lgbm_{}".format(i)] = pair_m["lgbm_model"].predict(df_probs_train_renta_2_complement[predictores])

df_probs_train_renta_2_complement["prob_cbtm"] = df_probs_train_renta_2_complement[[c for c in df_probs_train_renta_2_complement.columns if "prob_cbtm" in c]].mean(axis=1)
df_probs_train_renta_2_complement["prob_lgbm"] = df_probs_train_renta_2_complement[[c for c in df_probs_train_renta_2_complement.columns if "prob_lgbm" in c]].mean(axis=1)

df_probs_train_renta_2_complement = df_probs_train_renta_2_complement[['prediction_id', 'new_target', 'prob_cbtm', 'prob_lgbm']]
df_probs_train_renta_2_final = pd.concat([df_probs_train_renta_2, df_probs_train_renta_2_complement], sort=False).reset_index(drop=True)

Modelo 0
Modelo 1
Modelo 2
Modelo 3
Modelo 4


In [ ]:
df_test_opt_wic_h_v2[["prediction_id", "class"]].to_csv('baseline.csv', index=False)

------